メインの分析を補足するための予備分析をするファイル

In [2]:
import numpy as np
import pandas as pd
df = pd.read_csv('SSDSE-A-2024.csv', encoding = 'shift-jis')

In [3]:
#どんなモデルを作るのにも必要なデータ加工

new_header = df.iloc[1]  # ヘッダーに設定する行を抽出
df = df[2:]  # ヘッダー以外を除外
df.columns = new_header # ヘッダーを設定

df = df.drop(columns = '地域コード') # 不要な列を削除

In [4]:
#出生率の定義, 出生率を定義できない行の削除

fr = pd.DataFrame({
    'fertility_rate': pd.to_numeric(df['出生数'], errors='coerce') / pd.to_numeric(df['15〜64歳人口'], errors='coerce')
    })  # fertility rate の暫定的な定義

#inf_rows = fr[np.isinf(fr['fertility_rate'])] #403行目が無限大になる
fr['fertility_rate'] = fr['fertility_rate'].replace([np.inf, -np.inf], np.nan) #403行目を一旦NaNに置き換え
fr = fr.dropna(subset = ['fertility_rate']) #403行目を除去
df = df.drop(index = 403) #元データセットの403行目も使わないので除去

全国市区町村における毎年の15~64歳人口を, 既存データ(5年ごとの15~64歳人口データセット)から推定

転入者率と出生率の相関を確認

In [10]:
tr = pd.DataFrame({
    'transfer_rate' :
        pd.to_numeric(df['転入者数（日本人移動者）'], errors='coerce') / pd.to_numeric(df['日本人人口'], errors='coerce')
    }) # transfer rateを定義

correlation = tr['transfer_rate'].astype(float).corr(
    fr['fertility_rate'].astype(float)
) # trとfrの相関を計算

correlation

0.4109997214675287